In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
dataset = pd.read_csv("/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv")

In [4]:
dataset.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [5]:
# !pip install transformers

In [6]:
list(dataset)

['Unnamed: 0',
 'Clothing ID',
 'Age',
 'Title',
 'Review Text',
 'Rating',
 'Recommended IND',
 'Positive Feedback Count',
 'Division Name',
 'Department Name',
 'Class Name']

In [7]:
dataset = dataset.rename(columns={'Review Text': 'review', 'Recommended IND':'recommended'})
list(dataset)

['Unnamed: 0',
 'Clothing ID',
 'Age',
 'Title',
 'review',
 'Rating',
 'recommended',
 'Positive Feedback Count',
 'Division Name',
 'Department Name',
 'Class Name']

In [27]:
df = dataset.loc[:, ['review','recommended']].copy()

In [28]:
batch1 = df[:2000]

In [9]:
from transformers import DistilBertModel,DistilBertTokenizer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [23]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.device('cuda')

In [11]:
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
pretrained_weights = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(pretrained_weights)

In [29]:
#Tokenization:Convert words in 'review' column to numbers
# tokenized_reviews = df.review.apply(lambda x: tokenizer.encode(str(x),add_special_tokens=True))
tokenized_reviews1 = batch1.review.apply(lambda x: tokenizer.encode(str(x),add_special_tokens=True))

In [30]:
tokenized_reviews1

0       [101, 7078, 6919, 1011, 18848, 1998, 7916, 199...
1       [101, 2293, 2023, 4377, 999, 2009, 1005, 1055,...
2       [101, 1045, 2018, 2107, 2152, 8069, 2005, 2023...
3       [101, 1045, 2293, 1010, 2293, 1010, 2293, 2023...
4       [101, 2023, 3797, 2003, 2200, 4257, 17989, 200...
                              ...                        
1995    [101, 3430, 2003, 2524, 2000, 4929, 1037, 1165...
1996    [101, 2200, 3730, 8313, 1012, 2515, 4088, 2000...
1997    [101, 2122, 6312, 8549, 1999, 2035, 1996, 2157...
1998    [101, 3866, 1996, 2126, 2122, 2371, 2021, 2066...
1999    [101, 3376, 4377, 999, 3819, 3054, 3091, 4377,...
Name: review, Length: 2000, dtype: object

In [31]:
#Padding:To make all sentences of same length.This is only required for batch creation
'''First we need to find maximum length of senetence/review.'''
max_len = max(map(len,tokenized_reviews1))
padded_reviews = np.array([ i+[0]*(max_len-len(i))  for i in tokenized_reviews1])

In [32]:
#Masking:To tell DistilBert to ignore padding. This is called attention masking.Basically it'll put 1 where encoding is present and 0 for others.
attention_masked_reviews = np.where(padded_reviews!=0,1,0)
attention_masked_reviews.shape

(2000, 148)

In [16]:
np.array(attention_masked_reviews).shape

(23486, 162)

In [17]:
torch.cuda.current_device()

0

In [33]:
# Get last hidden states
input_ids = torch.tensor(padded_reviews)
attention_mask = torch.tensor(attention_masked_reviews)
with torch.no_grad():
  last_hidden_states =  model(input_ids,attention_mask=attention_mask)

In [39]:
#We'll save those in the features variable, as they'll serve as the features to our logitics regression model.
features = last_hidden_states[0][:,0,:].numpy()

#lables - Target 
labels = batch1['recommended']

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [43]:
from sklearn.linear_model import LogisticRegression

In [44]:
lr_clf = LogisticRegression(multi_class = 'ovr', C=1, solver='sag')
lr_clf.fit(train_features, train_labels)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1, multi_class='ovr', solver='sag')

In [45]:
# Model Evaluation 
lr_clf.score(test_features, test_labels)

0.876